https://github.com/dhammack/DSB2017/blob/master/scoring_code/score_ensemble2_final.py

Danniel applied data augmentation here, and he actually compute the max, and std on-line. But location features is computed from the Top_voxels files. 

In [3]:
import numpy as np
import pandas as pd
import os

In [99]:
PATH_RAW_ARRAY = "../../data/stage1_arrays/"
PATH_VOXELS = "../../data/stage1_TOP_voxels/"

patients = [f[4:] for f in os.listdir(PATH_VOXELS) if 'vox_' in f ]
print ("patient numbers: ", len(patients) )    

patient numbers:  222


In [98]:
from sklearn.cluster import DBSCAN
max_ct_count = []
all_features = []
for num, patient in enumerate(patients[:10]):
    patient_vox = np.load(os.path.join(PATH_VOXELS, 'vox_' + patient)) #voxels[filter]
    patient_locs = np.load(os.path.join(PATH_VOXELS, 'cents_' + patient))#locations[filter]
    patient_sizes = np.load(os.path.join(PATH_VOXELS, 'shapes_' + patient))#sizes[filter]
    patient_nodule_preds = np.load(os.path.join(PATH_VOXELS, 'preds_' + patient))
    max_ct_count.append(len(patient_nodule_preds[0,:]))
    
    xmax = np.max(patient_nodule_preds[0], axis=0)
    xsd = np.std(patient_nodule_preds[0], axis=0)
#     print ("max malignancy:", xmax)
#     print ("malignancy std:", xsd)
    
    normalized_locs = patient_locs.astype('float32') / patient_sizes.astype('float32')
#     print (normalized_locs)
    loc_from_malig = normalized_locs[ np.argmax(patient_nodule_preds[0],axis=0 )]
#     print ("normalized location from malig:", loc_from_malig)
    
    dist_mat = np.zeros((patient_locs.shape[0], patient_locs.shape[0]))
    for i,loc_a in enumerate(patient_locs):
        for j,loc_b in enumerate(patient_locs):
            dist_mat[i,j] = np.mean(np.abs(loc_a - loc_b))
#     print (dist_mat)

    dbs = DBSCAN(eps=60, min_samples=2, metric='precomputed', leaf_size=2).fit(dist_mat)
    num_clusters = np.max(dbs.labels_) + 1
    num_noise = (dbs.labels_ == -1).sum()
    print (" num_clusters", num_clusters, "\tnum_noise:", num_noise)
    
    #new feature: sum of malig_scores but normalizing by cluster.
    cluster_avgs = []
    for clusternum in range(num_clusters):
        cluster_avgs.append( patient_nodule_preds[0][dbs.labels_ == clusternum].mean())

    #now get the -1's
    for i,(clusterix,malig) in enumerate(zip(dbs.labels_,patient_nodule_preds[0])):
        if clusterix == -1:
            cluster_avgs.append(malig)

    weighted_sum_malig = np.sum(cluster_avgs)
    weighted_mean_malig = np.mean(cluster_avgs)

    #size of biggest cluster
    sizes = np.bincount(dbs.labels_[dbs.labels_ > 0])
    if len(sizes) > 0:
        maxsize = np.max(sizes)
    else:
        maxsize = 1
    n_nodules = float(patient_locs.shape[0])
    feats = (np.concatenate([(xmax, xsd),loc_from_malig, \
                           normalized_locs.std(axis=0), \
                           [float(num_clusters) / n_nodules, \
                            float(num_noise) / n_nodules, \
                            weighted_mean_malig, \
                            float(maxsize) / n_nodules]]))
    all_features.append(feats)
    X = np.stack(all_features)
    if num%10==0:
        print ("\nPatient:", patient[:-4])
        print (feats)

col=['max_malig', 'malig_xstd', 'locs_x', 'locs_y','locs_z', 
     'norm_locs_x','norm_locs_y','norm_locs_z','norm_#cluster',
     'norm_#noise','weighted_mean_malig', 'norm_maxsize' ]
df = pd.DataFrame(data=X,index=patients[:10], columns=col)
df.to_csv('feat_mat_v0.csv')

# print (max_ct_count)

 num_clusters 1 	num_noise: 0

Patient: c0f0eb84e70b19544943bed0ea6bd374
[ 0.67239404  0.04441824  0.33000001  0.37860084  0.88104087  0.23431544
  0.18727314  0.17379019  0.02        0.          0.54617685  0.02      ]
 num_clusters 1 	num_noise: 0
 num_clusters 1 	num_noise: 0
 num_clusters 1 	num_noise: 0
 num_clusters 1 	num_noise: 0
 num_clusters 1 	num_noise: 0
 num_clusters 1 	num_noise: 0
 num_clusters 1 	num_noise: 0
 num_clusters 1 	num_noise: 0
 num_clusters 1 	num_noise: 0


In [103]:
df_feature = pd.read_csv('feat_mat_v0.csv')
df_feature['id'] = df_feature['Unnamed: 0'].apply(lambda x: x.split('.')[0])
df_feature.head()

,Unnamed: 0,max_malig,malig_xstd,locs_x,locs_y,locs_z,norm_locs_x,norm_locs_y,norm_locs_z,norm_#cluster,norm_#noise,weighted_mean_malig,norm_maxsize,id
0,c0f0eb84e70b19544943bed0ea6bd374.npy,0.672394,0.044418,0.330000,0.378601,0.881041,0.234315,0.187273,0.173790,0.02,0.0,0.546177,0.02,c0f0eb84e70b19544943bed0ea6bd374
1,9065f2b133129c5747d42db18a424749.npy,0.701259,0.051872,0.779264,0.618834,0.886926,0.267581,0.142081,0.195229,0.02,0.0,0.542177,0.02,9065f2b133129c5747d42db18a424749
2,07b1defcfae5873ee1f03c90255eb170.npy,0.688366,0.046750,0.107383,0.379167,0.874016,0.259976,0.200964,0.237116,0.02,0.0,0.547058,0.02,07b1defcfae5873ee1f03c90255eb170
3,624a34fa8fd36847724e749877343847.npy,0.690876,0.050268,0.332237,0.770115,0.776224,0.285779,0.193638,0.227836,0.02,0.0,0.555996,0.02,624a34fa8fd36847724e749877343847
4,eb8d5136918d6859ca3cc3abafe369ac.npy,0.632769,0.044882,0.198795,0.660156,0.784810,0.272847,0.158534,0.193080,0.02,0.0,0.535779,0.02,eb8d5136918d6859ca3cc3abafe369ac
